## benzimidazole

In [675]:
from ase.io import read
from ase.visualize import view
xyz_path_complex = 'complexes/xyz/benzimidazole.xyz'
complex_atoms = read(xyz_path_complex)
view(complex_atoms, viewer='x3d')

In [672]:
xyz_path = 'complexes/xyz/benzimidazole_hydride.xyz'
complex_hydride_atoms = read(xyz_path)
# view(complex_hydride_atoms, viewer='x3d')

In [674]:
from rdkit import Chem
from rdkit.Chem import AllChem
import ase

smiles = "[nH]1cnc2ccccc12"
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)

# ---- Add 3D coords ----
AllChem.EmbedMolecule(mol, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(mol)

# ---- Your conversion function ----
def rdkit_mol_to_ase_atoms(rdkit_mol):
    conf = rdkit_mol.GetConformer()

    ase_atoms = ase.Atoms(
        numbers=[atom.GetAtomicNum() for atom in rdkit_mol.GetAtoms()],
        positions=conf.GetPositions()
    )
    return ase_atoms
ligand_atoms = rdkit_mol_to_ase_atoms(mol)
print(ligand_atoms)
view(ligand_atoms, viewer='x3d')

Atoms(symbols='NCNC6H6', pbc=False)


In [519]:
from ase import Atoms
from ase.optimize import FIRE
from ase.constraints import FixAtoms, FixCartesian
from fairchem.core import pretrained_mlip, FAIRChemCalculator
from ase.io import write
import numpy as np

# --- Step 1: Initialize FairChem calculator ---
predictor = pretrained_mlip.get_predict_unit("uma-s-1p1", device="cuda")
calc = FAIRChemCalculator(predictor, task_name="omol")
complex_atoms.calc = calc
# --- Step 2: Relax the complex ---
opt = FIRE(complex_atoms)
opt.run(fmax=0.1, steps=200)
E_complex = complex_atoms.get_potential_energy()
print(f"Relaxed complex energy: {E_complex:.3f} eV")
view(complex_atoms, viewer='x3d')

      Step     Time          Energy          fmax
FIRE:    0 19:27:28   -82388.633017        1.351882
FIRE:    1 19:27:28   -82388.767639        0.542354
FIRE:    2 19:27:28   -82388.764799        1.467831
FIRE:    3 19:27:28   -82388.803044        1.089336
FIRE:    4 19:27:29   -82388.842764        0.487977
FIRE:    5 19:27:29   -82388.852438        0.390984
FIRE:    6 19:27:29   -82388.854201        0.358284
FIRE:    7 19:27:29   -82388.857402        0.337410
FIRE:    8 19:27:29   -82388.861489        0.336484
FIRE:    9 19:27:29   -82388.865859        0.335427
FIRE:   10 19:27:29   -82388.870066        0.334244
FIRE:   11 19:27:29   -82388.873981        0.332840
FIRE:   12 19:27:29   -82388.877818        0.331051
FIRE:   13 19:27:29   -82388.882452        0.339421
FIRE:   14 19:27:30   -82388.888435        0.344374
FIRE:   15 19:27:30   -82388.896021        0.319695
FIRE:   16 19:27:30   -82388.904768        0.313332
FIRE:   17 19:27:30   -82388.913628        0.305653
FIRE:   18 19:

In [520]:
# --- Step 3: Energy of complex + Hydride ---
opt_ads = FIRE(complex_hydride_atoms)
complex_hydride_atoms.calc = calc
opt_ads.run(fmax=0.1, steps=200)
E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
print(f"Relaxed complex + hydride energy: {E_complex_hydride_atoms:.3f} eV")
view(complex_hydride_atoms, viewer='x3d')

      Step     Time          Energy          fmax
FIRE:    0 19:27:32   -72067.746395        1.204221
FIRE:    1 19:27:32   -72067.836889        0.559336
FIRE:    2 19:27:32   -72067.833993        1.281518
FIRE:    3 19:27:32   -72067.862376        0.920267
FIRE:    4 19:27:32   -72067.889868        0.353779
FIRE:    5 19:27:32   -72067.892876        0.413629
FIRE:    6 19:27:32   -72067.894227        0.377578
FIRE:    7 19:27:32   -72067.896626        0.319944
FIRE:    8 19:27:32   -72067.899574        0.288682
FIRE:    9 19:27:32   -72067.902505        0.279679
FIRE:   10 19:27:33   -72067.904993        0.268396
FIRE:   11 19:27:33   -72067.906932        0.254927
FIRE:   12 19:27:33   -72067.908575        0.255543
FIRE:   13 19:27:33   -72067.910518        0.304102
FIRE:   14 19:27:33   -72067.913191        0.307153
FIRE:   15 19:27:33   -72067.916748        0.254676
FIRE:   16 19:27:33   -72067.920689        0.158691
FIRE:   17 19:27:33   -72067.923979        0.139292
FIRE:   18 19:

In [521]:
# --- Step 5: get the energy removed ligand ---
opt_ligand = FIRE(ligand_atoms)
ligand_atoms.calc = calc
opt_ligand.run(fmax=0.1, steps=200)
E_ligand = ligand_atoms.get_potential_energy()
print(f"Relaxed ligand energy: {E_ligand:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 19:27:43   -10336.562875        3.858693
FIRE:    1 19:27:43   -10336.838833        1.434629
FIRE:    2 19:27:43   -10336.709255        3.269384
FIRE:    3 19:27:43   -10336.803771        2.053189
FIRE:    4 19:27:43   -10336.883067        0.557044
FIRE:    5 19:27:43   -10336.871876        1.267671
FIRE:    6 19:27:44   -10336.876076        1.171844
FIRE:    7 19:27:44   -10336.883267        0.986113
FIRE:    8 19:27:44   -10336.891387        0.722820
FIRE:    9 19:27:44   -10336.898142        0.405266
FIRE:   10 19:27:44   -10336.901758        0.296216
FIRE:   11 19:27:44   -10336.901907        0.366258
FIRE:   12 19:27:44   -10336.902001        0.360851
FIRE:   13 19:27:44   -10336.902184        0.350177
FIRE:   14 19:27:44   -10336.902447        0.334440
FIRE:   15 19:27:44   -10336.902777        0.314011
FIRE:   16 19:27:45   -10336.903159        0.289374
FIRE:   17 19:27:45   -10336.903576        0.265635
FIRE:   18 19:

In [522]:
# Define the H2 molecule (A standard bond length for H2 is ~0.74 Å)
h2_molecule = Atoms("H2", positions=[[0, 0, 0], [0.74, 0, 0]])
h2_molecule.calc = calc

# Relax the H2 molecule
opt_h2 = FIRE(h2_molecule)
# Use a high fmax and fewer steps as H2 relaxation is fast
opt_h2.run(fmax=0.01, steps=100) 

E_H2 = h2_molecule.get_potential_energy()
E_H = 0.5 * E_H2
print(f"Isolated H atom energy: {E_H:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 19:27:47      -31.589716        0.047594
FIRE:    1 19:27:47      -31.589744        0.012353
FIRE:    2 19:27:47      -31.589733        0.031729
FIRE:    3 19:27:47      -31.589737        0.025917
FIRE:    4 19:27:47      -31.589743        0.015339
FIRE:    5 19:27:47      -31.589746        0.001918
Isolated H atom energy: -15.795 eV


In [523]:
# --- Step 6: Compute adsorption energy ---
E_ads = (E_complex_hydride_atoms + E_ligand) - E_complex - E_H
print(f"Adsorption energy of H on Ni: {E_ads:.3f} eV")

Adsorption energy of H on Ni: -0.262 eV


## 2mercapto_6_methylpyridine

In [676]:
from ase.io import read
from ase.visualize import view
xyz_path_complex = 'complexes/xyz/2_mercapto_6_methylpyridine.xyz'
complex_atoms = read(xyz_path_complex)
# view(complex_atoms, viewer='x3d')

In [677]:
xyz_path = 'complexes/xyz/2_mercapto_6_methylpyridine_hydride.xyz'
complex_hydride_atoms = read(xyz_path)
# view(complex_hydride_atoms, viewer='x3d')

In [678]:
from rdkit import Chem
from rdkit.Chem import AllChem
import ase

smiles = "Cc1cccc([S-])n1"
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)

# ---- Add 3D coords ----
AllChem.EmbedMolecule(mol, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(mol)

# ---- Your conversion function ----
def rdkit_mol_to_ase_atoms(rdkit_mol):
    conf = rdkit_mol.GetConformer()

    ase_atoms = ase.Atoms(
        numbers=[atom.GetAtomicNum() for atom in rdkit_mol.GetAtoms()],
        positions=conf.GetPositions()
    )
    return ase_atoms
ligand_atoms = rdkit_mol_to_ase_atoms(mol)
print(ligand_atoms)
view(ligand_atoms, viewer='x3d')

Atoms(symbols='C6SNH6', pbc=False)


In [535]:
from ase import Atoms
from ase.optimize import FIRE
from ase.constraints import FixAtoms, FixCartesian
from fairchem.core import pretrained_mlip, FAIRChemCalculator
from ase.io import write
import numpy as np

# --- Step 1: Initialize FairChem calculator ---
predictor = pretrained_mlip.get_predict_unit("uma-s-1p1", device="cuda")
calc = FAIRChemCalculator(predictor, task_name="omol")
complex_atoms.calc = calc
# --- Step 2: Relax the complex ---
opt = FIRE(complex_atoms)
opt.run(fmax=0.1, steps=200)
E_complex = complex_atoms.get_potential_energy()
print(f"Relaxed complex energy: {E_complex:.3f} eV")
view(complex_atoms, viewer='x3d')

      Step     Time          Energy          fmax
FIRE:    0 19:32:49   -96977.613597        2.763788
FIRE:    1 19:32:49   -96977.905254        1.535992
FIRE:    2 19:32:49   -96978.101140        1.201376
FIRE:    3 19:32:49   -96978.182783        0.986749
FIRE:    4 19:32:50   -96978.207756        0.853204
FIRE:    5 19:32:50   -96978.247400        0.716460
FIRE:    6 19:32:50   -96978.287535        0.630787
FIRE:    7 19:32:50   -96978.319072        0.518162
FIRE:    8 19:32:50   -96978.342858        0.439604
FIRE:    9 19:32:50   -96978.365545        0.645939
FIRE:   10 19:32:50   -96978.392181        0.722229
FIRE:   11 19:32:50   -96978.427478        0.657711
FIRE:   12 19:32:50   -96978.470629        0.465248
FIRE:   13 19:32:51   -96978.518428        0.383257
FIRE:   14 19:32:51   -96978.566188        0.413336
FIRE:   15 19:32:51   -96978.610739        0.367071
FIRE:   16 19:32:51   -96978.654211        0.319223
FIRE:   17 19:32:51   -96978.697690        0.347532
FIRE:   18 19:

In [536]:
import numpy as np
from ase.optimize import FIRE
from ase.visualize import view

# ----------------------------------------------------------
# 1. Identify the metal atom
# ----------------------------------------------------------
metal_index = 0  # assume metal is atom 0
positions = complex_hydride_atoms.get_positions()
metal_pos = positions[metal_index]

# ----------------------------------------------------------
# 2. Find hydrogens bound to the metal
# ----------------------------------------------------------
hydride_indices = []
for i, atom in enumerate(complex_hydride_atoms):
    if atom.symbol == 'H':
        dist = np.linalg.norm(positions[i] - metal_pos)
        if dist < 2.0:    # typical M–H bond length cutoff
            hydride_indices.append(i)

print("Hydrides attached to metal:", hydride_indices)

# ----------------------------------------------------------
# 3. Remove one hydrogen (if there are two)
# ----------------------------------------------------------
if len(hydride_indices) > 1:
    remove_index = hydride_indices[1]  # remove second hydride
    complex_hydride_atoms.pop(remove_index)
    print(f"Removed hydrogen index {remove_index}")

# ----------------------------------------------------------
# 4. Run optimization with the calculator
# ----------------------------------------------------------
complex_hydride_atoms.calc = calc

opt = FIRE(complex_hydride_atoms)
opt.run(fmax=0.1, steps=200)

E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
print(f"Relaxed energy (single hydride): {E_relaxed:.3f} eV")
view(complex_hydride_atoms, viewer='x3d')


Hydrides attached to metal: [29, 30]
Removed hydrogen index 30
      Step     Time          Energy          fmax
FIRE:    0 19:32:53   -78347.716632        3.368866
FIRE:    1 19:32:53   -78348.056199        2.623341
FIRE:    2 19:32:53   -78348.298269        1.062060
FIRE:    3 19:32:53   -78348.320452        1.216154
FIRE:    4 19:32:53   -78348.346043        1.076753
FIRE:    5 19:32:54   -78348.386733        0.820299
FIRE:    6 19:32:54   -78348.427599        0.568481
FIRE:    7 19:32:54   -78348.457652        0.536869
FIRE:    8 19:32:54   -78348.475285        0.519217
FIRE:    9 19:32:54   -78348.486440        0.606435
FIRE:   10 19:32:54   -78348.498130        0.696225
FIRE:   11 19:32:54   -78348.516761        0.766204
FIRE:   12 19:32:54   -78348.544816        0.710025
FIRE:   13 19:32:54   -78348.580604        0.521197
FIRE:   14 19:32:55   -78348.617793        0.399172
FIRE:   15 19:32:55   -78348.647539        0.275671
FIRE:   16 19:32:55   -78348.667317        0.483118
FIR

In [538]:
# # --- Step 3: Energy of complex + Hydride ---
# opt_ads = FIRE(complex_hydride_atoms)
# complex_hydride_atoms.calc = calc
# opt_ads.run(fmax=0.1, steps=200)
# E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
# print(f"Relaxed complex + hydride energy: {E_complex_hydride_atoms:.3f} eV")
# view(complex_hydride_atoms, viewer='x3d')

In [539]:
# --- Step 5: get the energy removed ligand ---
opt_ligand = FIRE(ligand_atoms)
ligand_atoms.info.update({"charge": -1})
ligand_atoms.calc = calc
opt_ligand.run(fmax=0.1, steps=200)
E_ligand = ligand_atoms.get_potential_energy()
print(f"Relaxed ligand energy: {E_ligand:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 19:32:57   -18645.892820        2.435317
FIRE:    1 19:32:57   -18645.997520        1.217423
FIRE:    2 19:32:57   -18645.994786        1.483368
FIRE:    3 19:32:57   -18646.016698        1.063128
FIRE:    4 19:32:57   -18646.040824        0.679697
FIRE:    5 19:32:57   -18646.048138        0.440924
FIRE:    6 19:32:57   -18646.048761        0.409048
FIRE:    7 19:32:57   -18646.049898        0.347523
FIRE:    8 19:32:57   -18646.051363        0.309897
FIRE:    9 19:32:57   -18646.052939        0.304931
FIRE:   10 19:32:58   -18646.054448        0.297449
FIRE:   11 19:32:58   -18646.055804        0.286614
FIRE:   12 19:32:58   -18646.057040        0.271772
FIRE:   13 19:32:58   -18646.058391        0.250492
FIRE:   14 19:32:58   -18646.059999        0.248263
FIRE:   15 19:32:58   -18646.061914        0.223774
FIRE:   16 19:32:58   -18646.063960        0.147797
FIRE:   17 19:32:58   -18646.065734        0.152645
FIRE:   18 19:

In [540]:
# Define the H2 molecule (A standard bond length for H2 is ~0.74 Å)
h2_molecule = Atoms("H2", positions=[[0, 0, 0], [0.74, 0, 0]])
h2_molecule.calc = calc

# Relax the H2 molecule
opt_h2 = FIRE(h2_molecule)
# Use a high fmax and fewer steps as H2 relaxation is fast
opt_h2.run(fmax=0.01, steps=100) 

E_H2 = h2_molecule.get_potential_energy()
E_H = 0.5 * E_H2
print(f"Isolated H atom energy: {E_H:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 19:32:59      -31.589716        0.047594
FIRE:    1 19:33:00      -31.589744        0.012353
FIRE:    2 19:33:00      -31.589733        0.031727
FIRE:    3 19:33:00      -31.589737        0.025917
FIRE:    4 19:33:00      -31.589743        0.015339
FIRE:    5 19:33:00      -31.589746        0.001920
Isolated H atom energy: -15.795 eV


In [542]:
# --- Step 6: Compute adsorption energy ---
E_ads = (E_complex_hydride_atoms + E_ligand) - E_complex - E_H
print(f"Adsorption energy of H on Ni: {E_ads:.3f} eV")

Adsorption energy of H on Ni: -0.194 eV


## 6_mercaptonicotinonitrile

In [550]:
from ase.io import read
from ase.visualize import view
xyz_path_complex = 'complexes/xyz/6_mercaptonicotinonitrile.xyz'
complex_atoms = read(xyz_path_complex)
# view(complex_atoms, viewer='x3d')

In [551]:
xyz_path = 'complexes/xyz/6_mercaptonicotinonitrile_hydride.xyz'
complex_hydride_atoms = read(xyz_path)
# view(complex_hydride_atoms, viewer='x3d')

In [552]:
from ase import Atoms
from ase.optimize import FIRE
from ase.constraints import FixAtoms, FixCartesian
from fairchem.core import pretrained_mlip, FAIRChemCalculator
from ase.io import write
import numpy as np

# --- Step 1: Initialize FairChem calculator ---
predictor = pretrained_mlip.get_predict_unit("uma-s-1p1", device="cuda")
calc = FAIRChemCalculator(predictor, task_name="omol")
complex_atoms.calc = calc
# --- Step 2: Relax the complex ---
opt = FIRE(complex_atoms)
opt.run(fmax=0.1, steps=200)
E_complex = complex_atoms.get_potential_energy()
print(f"Relaxed complex energy: {E_complex:.3f} eV")
view(complex_atoms, viewer='x3d')

      Step     Time          Energy          fmax
FIRE:    0 19:34:51  -101294.972240       10.848197
FIRE:    1 19:34:51  -101297.796215        6.117557
FIRE:    2 19:34:51  -101299.140939        2.666852
FIRE:    3 19:34:51  -101299.546505        1.936128
FIRE:    4 19:34:51  -101299.632270        1.491448
FIRE:    5 19:34:51  -101299.710386        1.201913
FIRE:    6 19:34:51  -101299.780962        1.840360
FIRE:    7 19:34:52  -101299.886109        0.953519
FIRE:    8 19:34:52  -101299.964743        0.804357
FIRE:    9 19:34:52  -101300.021014        1.457276
FIRE:   10 19:34:52  -101300.103615        0.803249
FIRE:   11 19:34:52  -101300.182636        0.923957
FIRE:   12 19:34:52  -101300.274619        1.353607
FIRE:   13 19:34:52  -101300.400643        0.836092
FIRE:   14 19:34:52  -101300.514926        1.193262
FIRE:   15 19:34:52  -101300.653263        1.036654
FIRE:   16 19:34:53  -101300.802378        0.512388
FIRE:   17 19:34:53  -101300.920188        1.331785
FIRE:   18 19:

In [553]:
from rdkit import Chem
from rdkit.Chem import AllChem
import ase

smiles = "C1=CC(=S)[N-]C=C1C#N"
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)

# ---- Add 3D coords ----
AllChem.EmbedMolecule(mol, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(mol)

# ---- Your conversion function ----
def rdkit_mol_to_ase_atoms(rdkit_mol):
    conf = rdkit_mol.GetConformer()

    ase_atoms = ase.Atoms(
        numbers=[atom.GetAtomicNum() for atom in rdkit_mol.GetAtoms()],
        positions=conf.GetPositions()
    )
    return ase_atoms
ligand_atoms = rdkit_mol_to_ase_atoms(mol)
print(ligand_atoms)
view(ligand_atoms, viewer='x3d')

Atoms(symbols='C3SNC3NH3', pbc=False)


In [554]:
import numpy as np
from ase.optimize import FIRE
from ase.visualize import view

# ----------------------------------------------------------
# 1. Identify the metal atom
# ----------------------------------------------------------
metal_index = 0  # assume metal is atom 0
positions = complex_hydride_atoms.get_positions()
metal_pos = positions[metal_index]

# ----------------------------------------------------------
# 2. Find hydrogens bound to the metal
# ----------------------------------------------------------
hydride_indices = []
for i, atom in enumerate(complex_hydride_atoms):
    if atom.symbol == 'H':
        dist = np.linalg.norm(positions[i] - metal_pos)
        if dist < 2.0:    # typical M–H bond length cutoff
            hydride_indices.append(i)

print("Hydrides attached to metal:", hydride_indices)

# ----------------------------------------------------------
# 3. Remove one hydrogen (if there are two)
# ----------------------------------------------------------
if len(hydride_indices) > 1:
    remove_index = hydride_indices[1]  # remove second hydride
    complex_hydride_atoms.pop(remove_index)
    print(f"Removed hydrogen index {remove_index}")

# ----------------------------------------------------------
# 4. Run optimization with the calculator
# ----------------------------------------------------------
complex_hydride_atoms.calc = calc

opt = FIRE(complex_hydride_atoms)
opt.run(fmax=0.1, steps=200)

E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
print(f"Relaxed energy (single hydride): {E_relaxed:.3f} eV")
view(complex_hydride_atoms, viewer='x3d')


Hydrides attached to metal: [25, 26]
Removed hydrogen index 26
      Step     Time          Energy          fmax
FIRE:    0 19:35:01   -81225.773836       13.390210
FIRE:    1 19:35:01   -81228.237651        4.775156
FIRE:    2 19:35:01   -81228.867849        1.971797
FIRE:    3 19:35:01   -81228.928050        1.641556
FIRE:    4 19:35:02   -81228.991383        1.109942
FIRE:    5 19:35:02   -81229.036609        1.326654
FIRE:    6 19:35:02   -81229.082499        1.000789
FIRE:    7 19:35:02   -81229.127552        1.131342
FIRE:    8 19:35:02   -81229.177550        1.026424
FIRE:    9 19:35:02   -81229.235708        0.916454
FIRE:   10 19:35:02   -81229.288132        0.768265
FIRE:   11 19:35:02   -81229.347952        0.944360
FIRE:   12 19:35:02   -81229.417700        0.540675
FIRE:   13 19:35:03   -81229.473522        0.841597
FIRE:   14 19:35:03   -81229.539206        0.940543
FIRE:   15 19:35:03   -81229.616145        0.545884
FIRE:   16 19:35:03   -81229.673231        0.749422
FIR

In [548]:
# # --- Step 3: Energy of complex + Hydride ---
# opt_ads = FIRE(complex_hydride_atoms)
# complex_hydride_atoms.calc = calc
# opt_ads.run(fmax=0.05, steps=200)
# E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
# print(f"Relaxed complex + hydride energy: {E_complex_hydride_atoms:.3f} eV")
# view(complex_hydride_atoms, viewer='x3d')

## The hydrogens get desorped

In [555]:
# --- Step 5: get the energy removed ligand ---
opt_ligand = FIRE(ligand_atoms)
ligand_atoms.info.update({"charge": -1})
ligand_atoms.calc = calc
opt_ligand.run(fmax=0.1, steps=200)
E_ligand = ligand_atoms.get_potential_energy()
print(f"Relaxed ligand energy: {E_ligand:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 19:35:34   -20087.397354        2.810424
FIRE:    1 19:35:34   -20087.576368        0.898232
FIRE:    2 19:35:34   -20087.498796        2.941411
FIRE:    3 19:35:34   -20087.557726        1.962237
FIRE:    4 19:35:35   -20087.607661        0.555158
FIRE:    5 19:35:35   -20087.601367        1.099718
FIRE:    6 19:35:35   -20087.603804        1.033355
FIRE:    7 19:35:35   -20087.608011        0.904330
FIRE:    8 19:35:35   -20087.612856        0.720208
FIRE:    9 19:35:35   -20087.617067        0.493480
FIRE:   10 19:35:35   -20087.619648        0.248922
FIRE:   11 19:35:35   -20087.620330        0.263612
FIRE:   12 19:35:35   -20087.620386        0.262313
FIRE:   13 19:35:35   -20087.620495        0.259753
FIRE:   14 19:35:36   -20087.620654        0.255952
FIRE:   15 19:35:36   -20087.620856        0.250987
FIRE:   16 19:35:36   -20087.621093        0.244929
FIRE:   17 19:35:36   -20087.621355        0.237900
FIRE:   18 19:

In [556]:
# Define the H2 molecule (A standard bond length for H2 is ~0.74 Å)
h2_molecule = Atoms("H2", positions=[[0, 0, 0], [0.74, 0, 0]])
h2_molecule.calc = calc

# Relax the H2 molecule
opt_h2 = FIRE(h2_molecule)
# Use a high fmax and fewer steps as H2 relaxation is fast
opt_h2.run(fmax=0.01, steps=100) 

E_H2 = h2_molecule.get_potential_energy()
E_H = 0.5 * E_H2
print(f"Isolated H atom energy: {E_H:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 19:35:39      -31.589716        0.047594
FIRE:    1 19:35:39      -31.589744        0.012353
FIRE:    2 19:35:39      -31.589733        0.031727
FIRE:    3 19:35:39      -31.589737        0.025917
FIRE:    4 19:35:40      -31.589743        0.015339
FIRE:    5 19:35:40      -31.589746        0.001920
Isolated H atom energy: -15.795 eV


In [558]:
# --- Step 6: Compute adsorption energy ---
E_ads = (E_complex_hydride_atoms + E_ligand) - E_complex - E_H
print(f"Adsorption energy of H on Ni: {E_ads:.3f} eV")

Adsorption energy of H on Ni: -0.649 eV


## 3_bromo_6_mercaptopyridine

In [559]:
from ase.io import read
from ase.visualize import view
xyz_path_complex = 'complexes/xyz/3_bromo_6_mercaptopyridine.xyz'
complex_atoms = read(xyz_path_complex)
# view(complex_atoms, viewer='x3d')

In [560]:
xyz_path = 'complexes/xyz/3_bromo_6_mercaptopyridine_hydride.xyz'
complex_hydride_atoms = read(xyz_path)
# view(complex_hydride_atoms, viewer='x3d')

In [561]:
from ase import Atoms
from ase.optimize import FIRE
from ase.constraints import FixAtoms, FixCartesian
from fairchem.core import pretrained_mlip, FAIRChemCalculator
from ase.io import write
import numpy as np

# --- Step 1: Initialize FairChem calculator ---
predictor = pretrained_mlip.get_predict_unit("uma-s-1p1", device="cuda")
calc = FAIRChemCalculator(predictor, task_name="omol")
complex_atoms.calc = calc
# --- Step 2: Relax the complex ---
opt = FIRE(complex_atoms)
opt.run(fmax=0.1, steps=200)
E_complex = complex_atoms.get_potential_energy()
print(f"Relaxed complex energy: {E_complex:.3f} eV")
view(complex_atoms, viewer='x3d')

      Step     Time          Energy          fmax
FIRE:    0 19:36:31  -303842.046948       10.947763
FIRE:    1 19:36:31  -303844.966680        6.056933
FIRE:    2 19:36:31  -303846.256682        2.608957
FIRE:    3 19:36:31  -303846.664360        1.516570
FIRE:    4 19:36:31  -303846.569608        1.978639
FIRE:    5 19:36:31  -303846.642074        1.886199
FIRE:    6 19:36:32  -303846.760485        1.700233
FIRE:    7 19:36:32  -303846.892491        1.422179
FIRE:    8 19:36:32  -303847.020035        1.067903
FIRE:    9 19:36:32  -303847.136118        0.791253
FIRE:   10 19:36:32  -303847.230263        0.716322
FIRE:   11 19:36:32  -303847.295292        0.751231
FIRE:   12 19:36:32  -303847.348901        1.155876
FIRE:   13 19:36:32  -303847.415569        1.381503
FIRE:   14 19:36:32  -303847.507215        1.309309
FIRE:   15 19:36:32  -303847.620655        0.997405
FIRE:   16 19:36:33  -303847.766004        0.939205
FIRE:   17 19:36:33  -303847.922069        0.720738
FIRE:   18 19:

In [562]:
from rdkit import Chem
from rdkit.Chem import AllChem
import ase

smiles = "C1=CC(=S)[N-]C=C1Br"
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)

# ---- Add 3D coords ----
AllChem.EmbedMolecule(mol, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(mol)

# ---- Your conversion function ----
def rdkit_mol_to_ase_atoms(rdkit_mol):
    conf = rdkit_mol.GetConformer()

    ase_atoms = ase.Atoms(
        numbers=[atom.GetAtomicNum() for atom in rdkit_mol.GetAtoms()],
        positions=conf.GetPositions()
    )
    return ase_atoms
ligand_atoms = rdkit_mol_to_ase_atoms(mol)
print(ligand_atoms)
view(ligand_atoms, viewer='x3d')

Atoms(symbols='C3SNC2BrH3', pbc=False)


In [564]:
import numpy as np
from ase.optimize import FIRE
from ase.visualize import view

# ----------------------------------------------------------
# 1. Identify the metal atom
# ----------------------------------------------------------
metal_index = 0  # assume metal is atom 0
positions = complex_hydride_atoms.get_positions()
metal_pos = positions[metal_index]

# ----------------------------------------------------------
# 2. Find hydrogens bound to the metal
# ----------------------------------------------------------
hydride_indices = []
for i, atom in enumerate(complex_hydride_atoms):
    if atom.symbol == 'H':
        dist = np.linalg.norm(positions[i] - metal_pos)
        if dist < 2.0:    # typical M–H bond length cutoff
            hydride_indices.append(i)

print("Hydrides attached to metal:", hydride_indices)

# ----------------------------------------------------------
# 3. Remove one hydrogen (if there are two)
# ----------------------------------------------------------
if len(hydride_indices) > 1:
    remove_index = hydride_indices[1]  # remove second hydride
    complex_hydride_atoms.pop(remove_index)
    print(f"Removed hydrogen index {remove_index}")

# ----------------------------------------------------------
# 4. Run optimization with the calculator
# ----------------------------------------------------------
complex_hydride_atoms.calc = calc

opt = FIRE(complex_hydride_atoms)
opt.run(fmax=0.1, steps=200)

E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
print(f"Relaxed energy (single hydride): {E_relaxed:.3f} eV")
view(complex_hydride_atoms, viewer='x3d')


Hydrides attached to metal: [23, 24]
Removed hydrogen index 24
      Step     Time          Energy          fmax
FIRE:    0 19:36:58  -216257.164430       13.377674
FIRE:    1 19:36:58  -216259.652493        4.708706
FIRE:    2 19:36:58  -216260.252673        1.780843
FIRE:    3 19:36:58  -216260.292800        1.515581
FIRE:    4 19:36:58  -216260.350652        1.099722
FIRE:    5 19:36:58  -216260.403652        0.938084
FIRE:    6 19:36:58  -216260.448547        0.957493
FIRE:    7 19:36:59  -216260.494944        0.998095
FIRE:    8 19:36:59  -216260.546358        0.994285
FIRE:    9 19:36:59  -216260.598161        0.902565
FIRE:   10 19:36:59  -216260.654057        0.753080
FIRE:   11 19:36:59  -216260.717521        0.562622
FIRE:   12 19:36:59  -216260.783186        0.583725
FIRE:   13 19:36:59  -216260.843900        0.637640
FIRE:   14 19:36:59  -216260.913976        0.495715
FIRE:   15 19:36:59  -216260.985033        0.683211
FIRE:   16 19:36:59  -216261.055169        0.574048
FIR

In [563]:
# # --- Step 3: Energy of complex + Hydride ---
# opt_ads = FIRE(complex_hydride_atoms)
# complex_hydride_atoms.calc = calc
# opt_ads.run(fmax=0.1, steps=200)
# E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
# print(f"Relaxed complex + hydride energy: {E_complex_hydride_atoms:.3f} eV")
# view(complex_hydride_atoms, viewer='x3d')

## the hydrogens got detached

In [565]:
# --- Step 5: get the energy removed ligand ---
opt_ligand = FIRE(ligand_atoms)
ligand_atoms.info.update({"charge": -1})
ligand_atoms.calc = calc
opt_ligand.run(fmax=0.1, steps=200)
E_ligand = ligand_atoms.get_potential_energy()
print(f"Relaxed ligand energy: {E_ligand:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 19:37:14   -87602.760405        2.803504
FIRE:    1 19:37:14   -87602.916583        1.126023
FIRE:    2 19:37:14   -87602.884400        2.193429
FIRE:    3 19:37:14   -87602.917965        1.531588
FIRE:    4 19:37:15   -87602.950855        0.618696
FIRE:    5 19:37:15   -87602.954491        0.770109
FIRE:    6 19:37:15   -87602.955641        0.726102
FIRE:    7 19:37:15   -87602.957667        0.641117
FIRE:    8 19:37:15   -87602.960099        0.521314
FIRE:    9 19:37:15   -87602.962405        0.376473
FIRE:   10 19:37:15   -87602.964159        0.243265
FIRE:   11 19:37:15   -87602.965218        0.257722
FIRE:   12 19:37:15   -87602.965802        0.287806
FIRE:   13 19:37:15   -87602.966391        0.361262
FIRE:   14 19:37:16   -87602.967423        0.379282
FIRE:   15 19:37:16   -87602.969142        0.331072
FIRE:   16 19:37:16   -87602.971308        0.218595
FIRE:   17 19:37:16   -87602.973124        0.140687
FIRE:   18 19:

In [566]:
# Define the H2 molecule (A standard bond length for H2 is ~0.74 Å)
h2_molecule = Atoms("H2", positions=[[0, 0, 0], [0.74, 0, 0]])
h2_molecule.calc = calc

# Relax the H2 molecule
opt_h2 = FIRE(h2_molecule)
# Use a high fmax and fewer steps as H2 relaxation is fast
opt_h2.run(fmax=0.01, steps=100) 

E_H2 = h2_molecule.get_potential_energy()
E_H = 0.5 * E_H2
print(f"Isolated H atom energy: {E_H:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 19:37:18      -31.589716        0.047594
FIRE:    1 19:37:18      -31.589744        0.012353
FIRE:    2 19:37:18      -31.589733        0.031727
FIRE:    3 19:37:19      -31.589737        0.025916
FIRE:    4 19:37:19      -31.589743        0.015339
FIRE:    5 19:37:19      -31.589746        0.001920
Isolated H atom energy: -15.795 eV


In [568]:
# --- Step 6: Compute adsorption energy ---
E_ads = (E_complex_hydride_atoms + E_ligand) - E_complex - E_H
print(f"Adsorption energy of H on Ni: {E_ads:.3f} eV")

Adsorption energy of H on Ni: -0.297 eV


## 3_aminopyridine_2_1H_thione

In [569]:
from ase.io import read
from ase.visualize import view
xyz_path_complex = 'complexes/xyz/3_aminopyridine_2_1H_thione.xyz'
complex_atoms = read(xyz_path_complex)
# view(complex_atoms, viewer='x3d')

In [570]:
xyz_path = 'complexes/xyz/3_aminopyridine_2_1H_thione_hydride.xyz'
complex_hydride_atoms = read(xyz_path)
# view(complex_hydride_atoms, viewer='x3d')

In [571]:
from ase import Atoms
from ase.optimize import FIRE
from ase.constraints import FixAtoms, FixCartesian
from fairchem.core import pretrained_mlip, FAIRChemCalculator
from ase.io import write
import numpy as np

# --- Step 1: Initialize FairChem calculator ---
predictor = pretrained_mlip.get_predict_unit("uma-s-1p1", device="cuda")
calc = FAIRChemCalculator(predictor, task_name="omol")
complex_atoms.calc = calc
# --- Step 2: Relax the complex ---
opt = FIRE(complex_atoms)
opt.run(fmax=0.1, steps=200)
E_complex = complex_atoms.get_potential_energy()
print(f"Relaxed complex energy: {E_complex:.3f} eV")
view(complex_atoms, viewer='x3d')

      Step     Time          Energy          fmax
FIRE:    0 19:37:55   -98282.452753       11.019243
FIRE:    1 19:37:55   -98285.474189        6.152983
FIRE:    2 19:37:55   -98286.799888        2.564188
FIRE:    3 19:37:55   -98287.144245        1.473762
FIRE:    4 19:37:55   -98287.186537        1.385269
FIRE:    5 19:37:56   -98287.255891        1.212748
FIRE:    6 19:37:56   -98287.332560        0.968095
FIRE:    7 19:37:56   -98287.404447        0.682498
FIRE:    8 19:37:56   -98287.470275        0.664813
FIRE:    9 19:37:56   -98287.533246        0.648899
FIRE:   10 19:37:56   -98287.595936        0.727204
FIRE:   11 19:37:56   -98287.668409        0.933209
FIRE:   12 19:37:56   -98287.757122        0.973972
FIRE:   13 19:37:56   -98287.866308        0.798635
FIRE:   14 19:37:57   -98287.990243        0.461360
FIRE:   15 19:37:57   -98288.122494        0.531617
FIRE:   16 19:37:57   -98288.260851        0.522386
FIRE:   17 19:37:57   -98288.394436        0.685836
FIRE:   18 19:

In [572]:
from rdkit import Chem
from rdkit.Chem import AllChem
import ase

smiles = "C1=C[N-]C(=S)C(=C1)N"
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)

# ---- Add 3D coords ----
AllChem.EmbedMolecule(mol, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(mol)

# ---- Your conversion function ----
def rdkit_mol_to_ase_atoms(rdkit_mol):
    conf = rdkit_mol.GetConformer()

    ase_atoms = ase.Atoms(
        numbers=[atom.GetAtomicNum() for atom in rdkit_mol.GetAtoms()],
        positions=conf.GetPositions()
    )
    return ase_atoms
ligand_atoms = rdkit_mol_to_ase_atoms(mol)
print(ligand_atoms)
view(ligand_atoms, viewer='x3d')

Atoms(symbols='C2NCSC2NH5', pbc=False)


In [574]:
import numpy as np
from ase.optimize import FIRE
from ase.visualize import view

# ----------------------------------------------------------
# 1. Identify the metal atom
# ----------------------------------------------------------
metal_index = 0  # assume metal is atom 0
positions = complex_hydride_atoms.get_positions()
metal_pos = positions[metal_index]

# ----------------------------------------------------------
# 2. Find hydrogens bound to the metal
# ----------------------------------------------------------
hydride_indices = []
for i, atom in enumerate(complex_hydride_atoms):
    if atom.symbol == 'H':
        dist = np.linalg.norm(positions[i] - metal_pos)
        if dist < 2.0:    # typical M–H bond length cutoff
            hydride_indices.append(i)

print("Hydrides attached to metal:", hydride_indices)

# ----------------------------------------------------------
# 3. Remove one hydrogen (if there are two)
# ----------------------------------------------------------
if len(hydride_indices) > 1:
    remove_index = hydride_indices[1]  # remove second hydride
    complex_hydride_atoms.pop(remove_index)
    print(f"Removed hydrogen index {remove_index}")

# ----------------------------------------------------------
# 4. Run optimization with the calculator
# ----------------------------------------------------------
complex_hydride_atoms.calc = calc

opt = FIRE(complex_hydride_atoms)
opt.run(fmax=0.1, steps=200)

E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
print(f"Relaxed energy (single hydride): {E_relaxed:.3f} eV")
view(complex_hydride_atoms, viewer='x3d')


Hydrides attached to metal: [27, 28]
Removed hydrogen index 28
      Step     Time          Energy          fmax
FIRE:    0 19:38:28   -79217.423978       13.307969
FIRE:    1 19:38:29   -79220.003862        4.723763
FIRE:    2 19:38:29   -79220.646916        1.538682
FIRE:    3 19:38:29   -79220.687187        1.344955
FIRE:    4 19:38:29   -79220.746425        1.044088
FIRE:    5 19:38:29   -79220.802028        0.949407
FIRE:    6 19:38:29   -79220.849352        0.946914
FIRE:    7 19:38:29   -79220.898068        0.966390
FIRE:    8 19:38:29   -79220.953694        0.957628
FIRE:    9 19:38:29   -79221.012188        0.883367
FIRE:   10 19:38:29   -79221.075334        0.737641
FIRE:   11 19:38:29   -79221.144919        0.644393
FIRE:   12 19:38:30   -79221.221677        0.614971
FIRE:   13 19:38:30   -79221.300383        0.669493
FIRE:   14 19:38:30   -79221.386557        0.542625
FIRE:   15 19:38:30   -79221.475957        0.458175
FIRE:   16 19:38:30   -79221.555910        0.499039
FIR

In [573]:
# # --- Step 3: Energy of complex + Hydride ---
# opt_ads = FIRE(complex_hydride_atoms)
# complex_hydride_atoms.calc = calc
# opt_ads.run(fmax=0.1, steps=200)
# E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
# print(f"Relaxed complex + hydride energy: {E_complex_hydride_atoms:.3f} eV")
# view(complex_hydride_atoms, viewer='x3d')

## the hydrogens got desorped

In [575]:
# --- Step 5: get the energy removed ligand ---
opt_ligand = FIRE(ligand_atoms)
ligand_atoms.info.update({"charge": -1})
ligand_atoms.calc = calc
opt_ligand.run(fmax=0.1, steps=200)
E_ligand = ligand_atoms.get_potential_energy()
print(f"Relaxed ligand energy: {E_ligand:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 19:38:50   -19082.638623        2.958243
FIRE:    1 19:38:50   -19082.861478        1.240572
FIRE:    2 19:38:50   -19082.867168        2.064575
FIRE:    3 19:38:50   -19082.897882        1.483803
FIRE:    4 19:38:50   -19082.934950        0.869482
FIRE:    5 19:38:50   -19082.953129        0.647022
FIRE:    6 19:38:51   -19082.952397        1.466955
FIRE:    7 19:38:51   -19082.954899        1.386326
FIRE:    8 19:38:51   -19082.959403        1.230414
FIRE:    9 19:38:51   -19082.965033        1.009869
FIRE:   10 19:38:51   -19082.970778        0.740502
FIRE:   11 19:38:51   -19082.975772        0.583715
FIRE:   12 19:38:51   -19082.979595        0.563361
FIRE:   13 19:38:51   -19082.982462        0.542711
FIRE:   14 19:38:51   -19082.985350        0.520970
FIRE:   15 19:38:51   -19082.989238        0.499134
FIRE:   16 19:38:52   -19082.995049        0.521602
FIRE:   17 19:38:52   -19083.002985        0.456373
FIRE:   18 19:

In [576]:
# Define the H2 molecule (A standard bond length for H2 is ~0.74 Å)
h2_molecule = Atoms("H2", positions=[[0, 0, 0], [0.74, 0, 0]])
h2_molecule.calc = calc

# Relax the H2 molecule
opt_h2 = FIRE(h2_molecule)
# Use a high fmax and fewer steps as H2 relaxation is fast
opt_h2.run(fmax=0.01, steps=100) 

E_H2 = h2_molecule.get_potential_energy()
E_H = 0.5 * E_H2
print(f"Isolated H atom energy: {E_H:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 19:38:54      -31.589716        0.047594
FIRE:    1 19:38:54      -31.589744        0.012353
FIRE:    2 19:38:54      -31.589733        0.031727
FIRE:    3 19:38:54      -31.589737        0.025917
FIRE:    4 19:38:54      -31.589743        0.015339
FIRE:    5 19:38:54      -31.589746        0.001920
Isolated H atom energy: -15.795 eV


In [578]:
# --- Step 6: Compute adsorption energy ---
E_ads = (E_complex_hydride_atoms + E_ligand) - E_complex - E_H
print(f"Adsorption energy of H on Ni: {E_ads:.3f} eV")

Adsorption energy of H on Ni: -0.008 eV


## 5_chloropyridine_2_thiol

In [579]:
from ase.io import read
from ase.visualize import view
xyz_path_complex = 'complexes/xyz/5_chloropyridine_2_thiol.xyz'
complex_atoms = read(xyz_path_complex)
# view(complex_atoms, viewer='x3d')

In [580]:
xyz_path = 'complexes/xyz/5_chloropyridine_2_thiol_hydride.xyz'
complex_hydride_atoms = read(xyz_path)
# view(complex_hydride_atoms, viewer='x3d')

In [581]:
from ase import Atoms
from ase.optimize import FIRE
from ase.constraints import FixAtoms, FixCartesian
from fairchem.core import pretrained_mlip, FAIRChemCalculator
from ase.io import write
import numpy as np

# --- Step 1: Initialize FairChem calculator ---
predictor = pretrained_mlip.get_predict_unit("uma-s-1p1", device="cuda")
calc = FAIRChemCalculator(predictor, task_name="omol")
complex_atoms.calc = calc
# --- Step 2: Relax the complex ---
opt = FIRE(complex_atoms)
opt.run(fmax=0.1, steps=200)
E_complex = complex_atoms.get_potential_energy()
print(f"Relaxed complex energy: {E_complex:.3f} eV")
view(complex_atoms, viewer='x3d')

      Step     Time          Energy          fmax
FIRE:    0 19:39:58  -131287.371086        3.875773
FIRE:    1 19:39:58  -131287.842404        1.567345
FIRE:    2 19:39:58  -131287.978506        2.086096
FIRE:    3 19:39:58  -131288.048687        1.493270
FIRE:    4 19:39:58  -131288.120956        0.614466
FIRE:    5 19:39:59  -131288.144957        0.798890
FIRE:    6 19:39:59  -131288.148990        0.755725
FIRE:    7 19:39:59  -131288.156322        0.675143
FIRE:    8 19:39:59  -131288.165677        0.585772
FIRE:    9 19:39:59  -131288.175687        0.530883
FIRE:   10 19:39:59  -131288.185353        0.474859
FIRE:   11 19:39:59  -131288.194446        0.443825
FIRE:   12 19:39:59  -131288.203568        0.477175
FIRE:   13 19:39:59  -131288.214800        0.492700
FIRE:   14 19:39:59  -131288.229502        0.475168
FIRE:   15 19:40:00  -131288.248286        0.415363
FIRE:   16 19:40:00  -131288.270238        0.347742
FIRE:   17 19:40:00  -131288.293414        0.375620
FIRE:   18 19:

In [583]:
import numpy as np
from ase.optimize import FIRE
from ase.visualize import view

# ----------------------------------------------------------
# 1. Identify the metal atom
# ----------------------------------------------------------
metal_index = 0  # assume metal is atom 0
positions = complex_hydride_atoms.get_positions()
metal_pos = positions[metal_index]

# ----------------------------------------------------------
# 2. Find hydrogens bound to the metal
# ----------------------------------------------------------
hydride_indices = []
for i, atom in enumerate(complex_hydride_atoms):
    if atom.symbol == 'H':
        dist = np.linalg.norm(positions[i] - metal_pos)
        if dist < 2.0:    # typical M–H bond length cutoff
            hydride_indices.append(i)

print("Hydrides attached to metal:", hydride_indices)

# ----------------------------------------------------------
# 3. Remove one hydrogen (if there are two)
# ----------------------------------------------------------
if len(hydride_indices) > 1:
    remove_index = hydride_indices[1]  # remove second hydride
    complex_hydride_atoms.pop(remove_index)
    print(f"Removed hydrogen index {remove_index}")

# ----------------------------------------------------------
# 4. Run optimization with the calculator
# ----------------------------------------------------------
complex_hydride_atoms.calc = calc

opt = FIRE(complex_hydride_atoms)
opt.run(fmax=0.1, steps=200)

E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
print(f"Relaxed energy (single hydride): {E_relaxed:.3f} eV")
view(complex_hydride_atoms, viewer='x3d')


Hydrides attached to metal: [23, 24]
Removed hydrogen index 24
      Step     Time          Energy          fmax
FIRE:    0 19:40:13  -101220.846167        3.980609
FIRE:    1 19:40:13  -101221.345223        2.756135
FIRE:    2 19:40:13  -101221.535159        2.169731
FIRE:    3 19:40:13  -101221.587140        1.568848
FIRE:    4 19:40:13  -101221.640005        0.634582
FIRE:    5 19:40:13  -101221.655354        0.648979
FIRE:    6 19:40:13  -101221.657890        0.604303
FIRE:    7 19:40:13  -101221.662430        0.520033
FIRE:    8 19:40:13  -101221.668073        0.453317
FIRE:    9 19:40:13  -101221.673838        0.422990
FIRE:   10 19:40:14  -101221.678985        0.386448
FIRE:   11 19:40:14  -101221.683338        0.348925
FIRE:   12 19:40:14  -101221.687307        0.409503
FIRE:   13 19:40:14  -101221.692064        0.446846
FIRE:   14 19:40:14  -101221.698426        0.440366
FIRE:   15 19:40:14  -101221.706719        0.379752
FIRE:   16 19:40:14  -101221.716220        0.275705
FIR

In [582]:
# # --- Step 3: Energy of complex + Hydride ---
# opt_ads = FIRE(complex_hydride_atoms)
# complex_hydride_atoms.calc = calc
# opt_ads.run(fmax=0.1, steps=200)
# E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
# print(f"Relaxed complex + hydride energy: {E_complex_hydride_atoms:.3f} eV")
# view(complex_hydride_atoms, viewer='x3d')

In [584]:
from rdkit import Chem
from rdkit.Chem import AllChem
import ase

smiles = "C1=CC(=S)[N-]C=C1Cl"
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)

# ---- Add 3D coords ----
AllChem.EmbedMolecule(mol, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(mol)

# ---- Your conversion function ----
def rdkit_mol_to_ase_atoms(rdkit_mol):
    conf = rdkit_mol.GetConformer()

    ase_atoms = ase.Atoms(
        numbers=[atom.GetAtomicNum() for atom in rdkit_mol.GetAtoms()],
        positions=conf.GetPositions()
    )
    return ase_atoms
ligand_atoms = rdkit_mol_to_ase_atoms(mol)
print(ligand_atoms)
view(ligand_atoms, viewer='x3d')

Atoms(symbols='C3SNC2ClH3', pbc=False)


In [585]:
# --- Step 5: get the energy removed ligand ---
opt_ligand = FIRE(ligand_atoms)
ligand_atoms.info.update({"charge": -1})
ligand_atoms.calc = calc
opt_ligand.run(fmax=0.1, steps=200)
E_ligand = ligand_atoms.get_potential_energy()
print(f"Relaxed ligand energy: {E_ligand:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 19:40:23   -30082.756388        2.853337
FIRE:    1 19:40:23   -30082.915599        1.147514
FIRE:    2 19:40:23   -30082.882838        2.211563
FIRE:    3 19:40:23   -30082.917056        1.544364
FIRE:    4 19:40:23   -30082.950708        0.653811
FIRE:    5 19:40:24   -30082.954708        0.780572
FIRE:    6 19:40:24   -30082.955885        0.736272
FIRE:    7 19:40:24   -30082.957958        0.650753
FIRE:    8 19:40:24   -30082.960450        0.530260
FIRE:    9 19:40:24   -30082.962814        0.384755
FIRE:   10 19:40:24   -30082.964623        0.247109
FIRE:   11 19:40:24   -30082.965731        0.261344
FIRE:   12 19:40:24   -30082.966370        0.290555
FIRE:   13 19:40:24   -30082.967030        0.364591
FIRE:   14 19:40:24   -30082.968160        0.381812
FIRE:   15 19:40:25   -30082.970000        0.331428
FIRE:   16 19:40:25   -30082.972290        0.220928
FIRE:   17 19:40:25   -30082.974201        0.144668
FIRE:   18 19:

In [586]:
# Define the H2 molecule (A standard bond length for H2 is ~0.74 Å)
h2_molecule = Atoms("H2", positions=[[0, 0, 0], [0.74, 0, 0]])
h2_molecule.calc = calc

# Relax the H2 molecule
opt_h2 = FIRE(h2_molecule)
# Use a high fmax and fewer steps as H2 relaxation is fast
opt_h2.run(fmax=0.01, steps=100) 

E_H2 = h2_molecule.get_potential_energy()
E_H = 0.5 * E_H2
print(f"Isolated H atom energy: {E_H:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 19:40:28      -31.589716        0.047594
FIRE:    1 19:40:28      -31.589744        0.012353
FIRE:    2 19:40:28      -31.589733        0.031729
FIRE:    3 19:40:28      -31.589737        0.025917
FIRE:    4 19:40:29      -31.589743        0.015339
FIRE:    5 19:40:29      -31.589746        0.001918
Isolated H atom energy: -15.795 eV


In [588]:
# --- Step 6: Compute adsorption energy ---
E_ads = (E_complex_hydride_atoms + E_ligand) - E_complex - E_H
print(f"Adsorption energy of H on Ni: {E_ads:.3f} eV")

Adsorption energy of H on Ni: -0.374 eV


## 3_aminopyridine_2_1H_thione

In [589]:
from ase.io import read
from ase.visualize import view
xyz_path_complex = 'complexes/xyz/3_aminopyridine_2_1H_thione.xyz'
complex_atoms = read(xyz_path_complex)
# view(complex_atoms, viewer='x3d')

In [591]:
xyz_path = 'complexes/xyz/3_aminopyridine_2_1H_thione_hydride.xyz'
complex_hydride_atoms = read(xyz_path)
# view(complex_hydride_atoms, viewer='x3d')

In [592]:
from ase import Atoms
from ase.optimize import FIRE
from ase.constraints import FixAtoms, FixCartesian
from fairchem.core import pretrained_mlip, FAIRChemCalculator
from ase.io import write
import numpy as np

# --- Step 1: Initialize FairChem calculator ---
predictor = pretrained_mlip.get_predict_unit("uma-s-1p1", device="cuda")
calc = FAIRChemCalculator(predictor, task_name="omol")
complex_atoms.calc = calc
# --- Step 2: Relax the complex ---
opt = FIRE(complex_atoms)
opt.run(fmax=0.1, steps=200)
E_complex = complex_atoms.get_potential_energy()
print(f"Relaxed complex energy: {E_complex:.3f} eV")
view(complex_atoms, viewer='x3d')

      Step     Time          Energy          fmax
FIRE:    0 19:41:01   -98282.452753       11.019245
FIRE:    1 19:41:01   -98285.474189        6.152984
FIRE:    2 19:41:02   -98286.799888        2.564188
FIRE:    3 19:41:02   -98287.144247        1.473761
FIRE:    4 19:41:02   -98287.186535        1.385269
FIRE:    5 19:41:02   -98287.255892        1.212747
FIRE:    6 19:41:02   -98287.332559        0.968095
FIRE:    7 19:41:02   -98287.404447        0.682497
FIRE:    8 19:41:02   -98287.470275        0.664813
FIRE:    9 19:41:02   -98287.533245        0.648899
FIRE:   10 19:41:02   -98287.595936        0.727205
FIRE:   11 19:41:02   -98287.668408        0.933214
FIRE:   12 19:41:03   -98287.757122        0.973966
FIRE:   13 19:41:03   -98287.866308        0.798634
FIRE:   14 19:41:03   -98287.990243        0.461359
FIRE:   15 19:41:03   -98288.122493        0.531626
FIRE:   16 19:41:03   -98288.260852        0.522386
FIRE:   17 19:41:03   -98288.394435        0.685835
FIRE:   18 19:

In [594]:
from rdkit import Chem
from rdkit.Chem import AllChem
import ase

smiles = "C1=C[N-]C(=S)C(=C1)N"
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)

# ---- Add 3D coords ----
AllChem.EmbedMolecule(mol, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(mol)

# ---- Your conversion function ----
def rdkit_mol_to_ase_atoms(rdkit_mol):
    conf = rdkit_mol.GetConformer()

    ase_atoms = ase.Atoms(
        numbers=[atom.GetAtomicNum() for atom in rdkit_mol.GetAtoms()],
        positions=conf.GetPositions()
    )
    return ase_atoms
ligand_atoms = rdkit_mol_to_ase_atoms(mol)
print(ligand_atoms)
view(ligand_atoms, viewer='x3d')

Atoms(symbols='C2NCSC2NH5', pbc=False)


In [596]:
import numpy as np
from ase.optimize import FIRE
from ase.visualize import view

# ----------------------------------------------------------
# 1. Identify the metal atom
# ----------------------------------------------------------
metal_index = 0  # assume metal is atom 0
positions = complex_hydride_atoms.get_positions()
metal_pos = positions[metal_index]

# ----------------------------------------------------------
# 2. Find hydrogens bound to the metal
# ----------------------------------------------------------
hydride_indices = []
for i, atom in enumerate(complex_hydride_atoms):
    if atom.symbol == 'H':
        dist = np.linalg.norm(positions[i] - metal_pos)
        if dist < 2.0:    # typical M–H bond length cutoff
            hydride_indices.append(i)

print("Hydrides attached to metal:", hydride_indices)

# ----------------------------------------------------------
# 3. Remove one hydrogen (if there are two)
# ----------------------------------------------------------
if len(hydride_indices) > 1:
    remove_index = hydride_indices[1]  # remove second hydride
    complex_hydride_atoms.pop(remove_index)
    print(f"Removed hydrogen index {remove_index}")

# ----------------------------------------------------------
# 4. Run optimization with the calculator
# ----------------------------------------------------------
complex_hydride_atoms.calc = calc

opt = FIRE(complex_hydride_atoms)
opt.run(fmax=0.1, steps=200)

E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
print(f"Relaxed energy (single hydride): {E_relaxed:.3f} eV")
view(complex_hydride_atoms, viewer='x3d')

Hydrides attached to metal: [27, 28]
Removed hydrogen index 28
      Step     Time          Energy          fmax
FIRE:    0 19:41:40   -79217.423977       13.307971
FIRE:    1 19:41:40   -79220.003862        4.723761
FIRE:    2 19:41:41   -79220.646916        1.538682
FIRE:    3 19:41:41   -79220.687187        1.344954
FIRE:    4 19:41:41   -79220.746424        1.044088
FIRE:    5 19:41:41   -79220.802030        0.949408
FIRE:    6 19:41:41   -79220.849353        0.946914
FIRE:    7 19:41:41   -79220.898066        0.966390
FIRE:    8 19:41:41   -79220.953695        0.957628
FIRE:    9 19:41:41   -79221.012189        0.883366
FIRE:   10 19:41:41   -79221.075334        0.737640
FIRE:   11 19:41:41   -79221.144919        0.644393
FIRE:   12 19:41:42   -79221.221677        0.614974
FIRE:   13 19:41:42   -79221.300381        0.669496
FIRE:   14 19:41:42   -79221.386555        0.542622
FIRE:   15 19:41:42   -79221.475957        0.458180
FIRE:   16 19:41:42   -79221.555910        0.499049
FIR

In [595]:
# # --- Step 3: Energy of complex + Hydride ---
# opt_ads = FIRE(complex_hydride_atoms)
# complex_hydride_atoms.calc = calc
# opt_ads.run(fmax=0.1, steps=200)
# E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
# print(f"Relaxed complex + hydride energy: {E_complex_hydride_atoms:.3f} eV")
# view(complex_hydride_atoms, viewer='x3d')

## the hydrogen got desorped

In [597]:
# --- Step 5: get the energy removed ligand ---
opt_ligand = FIRE(ligand_atoms)
ligand_atoms.info.update({"charge": -1})
ligand_atoms.calc = calc
opt_ligand.run(fmax=0.1, steps=200)
E_ligand = ligand_atoms.get_potential_energy()
print(f"Relaxed ligand energy: {E_ligand:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 19:42:02   -19082.638622        2.958235
FIRE:    1 19:42:02   -19082.861478        1.240580
FIRE:    2 19:42:02   -19082.867168        2.064568
FIRE:    3 19:42:02   -19082.897882        1.483790
FIRE:    4 19:42:02   -19082.934951        0.869484
FIRE:    5 19:42:02   -19082.953129        0.647010
FIRE:    6 19:42:02   -19082.952397        1.466954
FIRE:    7 19:42:02   -19082.954899        1.386322
FIRE:    8 19:42:02   -19082.959403        1.230407
FIRE:    9 19:42:02   -19082.965033        1.009853
FIRE:   10 19:42:02   -19082.970779        0.740509
FIRE:   11 19:42:03   -19082.975773        0.583714
FIRE:   12 19:42:03   -19082.979596        0.563363
FIRE:   13 19:42:03   -19082.982463        0.542703
FIRE:   14 19:42:03   -19082.985351        0.520969
FIRE:   15 19:42:03   -19082.989239        0.499136
FIRE:   16 19:42:03   -19082.995049        0.521587
FIRE:   17 19:42:03   -19083.002985        0.456372
FIRE:   18 19:

In [598]:
# Define the H2 molecule (A standard bond length for H2 is ~0.74 Å)
h2_molecule = Atoms("H2", positions=[[0, 0, 0], [0.74, 0, 0]])
h2_molecule.calc = calc

# Relax the H2 molecule
opt_h2 = FIRE(h2_molecule)
# Use a high fmax and fewer steps as H2 relaxation is fast
opt_h2.run(fmax=0.01, steps=100) 

E_H2 = h2_molecule.get_potential_energy()
E_H = 0.5 * E_H2
print(f"Isolated H atom energy: {E_H:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 19:42:07      -31.589716        0.047594
FIRE:    1 19:42:07      -31.589744        0.012353
FIRE:    2 19:42:08      -31.589733        0.031727
FIRE:    3 19:42:08      -31.589737        0.025917
FIRE:    4 19:42:08      -31.589743        0.015339
FIRE:    5 19:42:08      -31.589746        0.001920
Isolated H atom energy: -15.795 eV


In [600]:
# --- Step 6: Compute adsorption energy ---
E_ads = (E_complex_hydride_atoms + E_ligand) - E_complex - E_H
print(f"Adsorption energy of H on Ni: {E_ads:.3f} eV")

Adsorption energy of H on Ni: -0.008 eV


## 3-5-benzenetricarboxylicacid

## You have to visualize the structures after relaxation!!

In [601]:
from ase.io import read
from ase.visualize import view
xyz_path_complex = 'savs_xyz_files/ni_oct_2_1-3-5-benzenetricarboxylicacid_3_s_1_conf_1.xyz'
complex_atoms = read(xyz_path_complex)
# view(complex_atoms, viewer='x3d')

In [602]:
xyz_path = 'savs_xyz_files/ni_oct_2_1-3-5-benzenetricarboxylicacid_2_hydride_2_s_1_conf_1.xyz'
complex_hydride_atoms = read(xyz_path)
# view(complex_hydride_atoms, viewer='x3d')

$$\Delta E_{ads} = (E_{complex+H}+E_{separate ligand}) - E_{complex} - E_H$$
### We need to add the energy of the separate ligand because in the hydride complex, we remove a ligand to add the hydrogen.

In [603]:
from rdkit import Chem
from rdkit.Chem import AllChem
import ase

smiles = "[O-]C(=O)c1cc(cc(c1)C(O)=O)C(O)=O"
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)

# ---- Add 3D coords ----
AllChem.EmbedMolecule(mol, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(mol)

# ---- Your conversion function ----
def rdkit_mol_to_ase_atoms(rdkit_mol):
    conf = rdkit_mol.GetConformer()

    ase_atoms = ase.Atoms(
        numbers=[atom.GetAtomicNum() for atom in rdkit_mol.GetAtoms()],
        positions=conf.GetPositions()
    )
    return ase_atoms
ligand_atoms = rdkit_mol_to_ase_atoms(mol)
print(ligand_atoms)
view(ligand_atoms, viewer='x3d')

Atoms(symbols='OCOC7O2CO2H5', pbc=False)


## Compute the Energy of the Complex

In [604]:
from ase import Atoms
from ase.optimize import FIRE
from ase.constraints import FixAtoms, FixCartesian
from fairchem.core import pretrained_mlip, FAIRChemCalculator
from ase.io import write
import numpy as np

# --- Step 1: Initialize FairChem calculator ---
predictor = pretrained_mlip.get_predict_unit("uma-s-1p1", device="cuda")
calc = FAIRChemCalculator(predictor, task_name="omol")
complex_atoms.calc = calc
# --- Step 2: Relax the complex ---
opt = FIRE(complex_atoms)
opt.run(fmax=0.1, steps=200)
E_complex = complex_atoms.get_potential_energy()
print(f"Relaxed complex energy: {E_complex:.3f} eV")
view(complex_atoms, viewer='x3d')

      Step     Time          Energy          fmax
FIRE:    0 19:43:00  -106136.578861       18.065714
FIRE:    1 19:43:00  -106141.525668       10.055770
FIRE:    2 19:43:01  -106143.554303        4.003893
FIRE:    3 19:43:01  -106143.774230        2.874484
FIRE:    4 19:43:01  -106143.918841        2.543930
FIRE:    5 19:43:01  -106144.123164        2.080603
FIRE:    6 19:43:01  -106144.280322        1.462456
FIRE:    7 19:43:01  -106144.348995        1.673765
FIRE:    8 19:43:01  -106144.382833        2.188249
FIRE:    9 19:43:01  -106144.398279        2.060861
FIRE:   10 19:43:01  -106144.425456        1.824487
FIRE:   11 19:43:01  -106144.458154        1.513813
FIRE:   12 19:43:02  -106144.489750        1.177883
FIRE:   13 19:43:02  -106144.515310        0.882913
FIRE:   14 19:43:02  -106144.533200        0.712040
FIRE:   15 19:43:02  -106144.545546        0.746096
FIRE:   16 19:43:02  -106144.557693        0.879563
FIRE:   17 19:43:02  -106144.574311        0.923791
FIRE:   18 19:

In [605]:
# # --- Step 3: Energy of complex + Hydride ---
# opt_ads = FIRE(complex_hydride_atoms)
# complex_hydride_atoms.calc = calc
# opt_ads.run(fmax=0.1, steps=200)
# E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
# print(f"Relaxed complex + hydride energy: {E_complex_hydride_atoms:.3f} eV")
# view(complex_hydride_atoms, viewer='x3d')

In [606]:
import numpy as np
from ase.optimize import FIRE
from ase.visualize import view

# ----------------------------------------------------------
# 1. Identify the metal atom
# ----------------------------------------------------------
metal_index = 0  # assume metal is atom 0
positions = complex_hydride_atoms.get_positions()
metal_pos = positions[metal_index]

# ----------------------------------------------------------
# 2. Find hydrogens bound to the metal
# ----------------------------------------------------------
hydride_indices = []
for i, atom in enumerate(complex_hydride_atoms):
    if atom.symbol == 'H':
        dist = np.linalg.norm(positions[i] - metal_pos)
        if dist < 2.0:    # typical M–H bond length cutoff
            hydride_indices.append(i)

print("Hydrides attached to metal:", hydride_indices)

# ----------------------------------------------------------
# 3. Remove one hydrogen (if there are two)
# ----------------------------------------------------------
if len(hydride_indices) > 1:
    remove_index = hydride_indices[1]  # remove second hydride
    complex_hydride_atoms.pop(remove_index)
    print(f"Removed hydrogen index {remove_index}")

# ----------------------------------------------------------
# 4. Run optimization with the calculator
# ----------------------------------------------------------
complex_hydride_atoms.calc = calc

opt = FIRE(complex_hydride_atoms)
opt.run(fmax=0.1, steps=200)

E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
print(f"Relaxed energy (single hydride): {E_relaxed:.3f} eV")
view(complex_hydride_atoms, viewer='x3d')

Hydrides attached to metal: [41, 42]
Removed hydrogen index 42
      Step     Time          Energy          fmax
FIRE:    0 19:43:22   -84452.338911       19.914417
FIRE:    1 19:43:22   -84456.701717        7.747941
FIRE:    2 19:43:22   -84457.933125        2.369414
FIRE:    3 19:43:22   -84457.686382        3.496286
FIRE:    4 19:43:22   -84457.857079        2.863263
FIRE:    5 19:43:22   -84458.086611        2.183053
FIRE:    6 19:43:22   -84458.234863        1.543475
FIRE:    7 19:43:22   -84458.266856        2.534586
FIRE:    8 19:43:22   -84458.281007        2.337770
FIRE:    9 19:43:22   -84458.305276        1.975412
FIRE:   10 19:43:23   -84458.333162        1.504557
FIRE:   11 19:43:23   -84458.358160        1.003424
FIRE:   12 19:43:23   -84458.376142        0.695900
FIRE:   13 19:43:23   -84458.386996        0.879024
FIRE:   14 19:43:23   -84458.394213        1.207685
FIRE:   15 19:43:23   -84458.403076        1.426647
FIRE:   16 19:43:23   -84458.417641        1.475043
FIR

In [608]:
# --- Step 5: get the energy removed ligand ---
opt_ligand = FIRE(ligand_atoms)
ligand_atoms.info.update({"charge": -1})
ligand_atoms.calc = calc
opt_ligand.run(fmax=0.1, steps=200)
E_ligand = ligand_atoms.get_potential_energy()
print(f"Relaxed ligand energy: {E_ligand:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 19:43:52   -21702.548731        0.055599
Relaxed ligand energy: -21702.549 eV


## Compute the energy of the Hydrogen atom

In [609]:
# Define the H2 molecule (A standard bond length for H2 is ~0.74 Å)
h2_molecule = Atoms("H2", positions=[[0, 0, 0], [0.74, 0, 0]])
h2_molecule.calc = calc

# Relax the H2 molecule
opt_h2 = FIRE(h2_molecule)
# Use a high fmax and fewer steps as H2 relaxation is fast
opt_h2.run(fmax=0.01, steps=100) 

E_H2 = h2_molecule.get_potential_energy()
E_H = 0.5 * E_H2
print(f"Isolated H atom energy: {E_H:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 19:43:55      -31.589716        0.047594
FIRE:    1 19:43:55      -31.589744        0.012353
FIRE:    2 19:43:55      -31.589733        0.031727
FIRE:    3 19:43:55      -31.589737        0.025916
FIRE:    4 19:43:55      -31.589743        0.015339
FIRE:    5 19:43:55      -31.589746        0.001920
Isolated H atom energy: -15.795 eV


In [611]:
# --- Step 6: Compute adsorption energy ---
E_ads = (E_complex_hydride_atoms + E_ligand) - E_complex - E_H
print(f"Adsorption energy of H on Ni: {E_ads:.3f} eV")

Adsorption energy of H on Ni: -0.438 eV


## 5,7dichloro8hydroxyquinoline

In [612]:
from ase.io import read
from ase.visualize import view
xyz_path_complex = 'savs_xyz_files/ni_oct_2_57dichloro8hydroxyquinoline_3_s_1_conf_1.xyz'
complex_atoms = read(xyz_path_complex)
# view(complex_atoms, viewer='x3d')

In [613]:
xyz_path = 'savs_xyz_files/ni_oct_2_57dichloro8hydroxyquinoline_2_hydride_2_s_1_conf_1.xyz'
complex_hydride_atoms = read(xyz_path)
# view(complex_hydride_atoms, viewer='x3d')

In [614]:
from rdkit import Chem
from rdkit.Chem import AllChem
import ase

smiles = "[O-]c1c(Cl)cc(Cl)c2cccnc12"
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)

# ---- Add 3D coords ----
AllChem.EmbedMolecule(mol, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(mol)

# ---- Your conversion function ----
def rdkit_mol_to_ase_atoms(rdkit_mol):
    conf = rdkit_mol.GetConformer()

    ase_atoms = ase.Atoms(
        numbers=[atom.GetAtomicNum() for atom in rdkit_mol.GetAtoms()],
        positions=conf.GetPositions()
    )
    return ase_atoms
ligand_atoms = rdkit_mol_to_ase_atoms(mol)
print(ligand_atoms)
# view(ligand_atoms, viewer='x3d')

Atoms(symbols='OC2ClC2ClC4NCH4', pbc=False)


In [615]:
from ase import Atoms
from ase.optimize import FIRE
from ase.constraints import FixAtoms, FixCartesian
from fairchem.core import pretrained_mlip, FAIRChemCalculator
from ase.io import write
import numpy as np

# --- Step 1: Initialize FairChem calculator ---
predictor = pretrained_mlip.get_predict_unit("uma-s-1p1", device="cuda")
calc = FAIRChemCalculator(predictor, task_name="omol")
complex_atoms.calc = calc
# --- Step 2: Relax the complex ---
opt = FIRE(complex_atoms)
opt.run(fmax=0.1, steps=200)
E_complex = complex_atoms.get_potential_energy()
print(f"Relaxed complex energy: {E_complex:.3f} eV")
view(complex_atoms, viewer='x3d')

      Step     Time          Energy          fmax
FIRE:    0 19:45:45  -154985.657390        2.346748
FIRE:    1 19:45:45  -154986.047947        1.052985
FIRE:    2 19:45:45  -154985.986329        1.984407
FIRE:    3 19:45:45  -154986.079671        1.453517
FIRE:    4 19:45:45  -154986.179516        0.712974
FIRE:    5 19:45:45  -154986.205649        0.627835
FIRE:    6 19:45:45  -154986.208224        0.594261
FIRE:    7 19:45:45  -154986.212910        0.529508
FIRE:    8 19:45:46  -154986.218899        0.438381
FIRE:    9 19:45:46  -154986.225307        0.376547
FIRE:   10 19:45:46  -154986.231428        0.363160
FIRE:   11 19:45:46  -154986.237011        0.347244
FIRE:   12 19:45:46  -154986.242318        0.329075
FIRE:   13 19:45:46  -154986.248505        0.324095
FIRE:   14 19:45:46  -154986.256348        0.310045
FIRE:   15 19:45:46  -154986.266281        0.289325
FIRE:   16 19:45:46  -154986.277769        0.282640
FIRE:   17 19:45:46  -154986.289294        0.273241
FIRE:   18 19:

In [616]:
# # --- Step 3: Energy of complex + Hydride ---
# opt_ads = FIRE(complex_hydride_atoms)
# complex_hydride_atoms.calc = calc
# opt_ads.run(fmax=0.1, steps=200)
# E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
# print(f"Relaxed complex + hydride energy: {E_complex_hydride_atoms:.3f} eV")
# view(complex_hydride_atoms, viewer='x3d')

In [617]:
import numpy as np
from ase.optimize import FIRE
from ase.visualize import view

# ----------------------------------------------------------
# 1. Identify the metal atom
# ----------------------------------------------------------
metal_index = 0  # assume metal is atom 0
positions = complex_hydride_atoms.get_positions()
metal_pos = positions[metal_index]

# ----------------------------------------------------------
# 2. Find hydrogens bound to the metal
# ----------------------------------------------------------
hydride_indices = []
for i, atom in enumerate(complex_hydride_atoms):
    if atom.symbol == 'H':
        dist = np.linalg.norm(positions[i] - metal_pos)
        if dist < 2.0:    # typical M–H bond length cutoff
            hydride_indices.append(i)

print("Hydrides attached to metal:", hydride_indices)

# ----------------------------------------------------------
# 3. Remove one hydrogen (if there are two)
# ----------------------------------------------------------
if len(hydride_indices) > 1:
    remove_index = hydride_indices[1]  # remove second hydride
    complex_hydride_atoms.pop(remove_index)
    print(f"Removed hydrogen index {remove_index}")

# ----------------------------------------------------------
# 4. Run optimization with the calculator
# ----------------------------------------------------------
complex_hydride_atoms.calc = calc

opt = FIRE(complex_hydride_atoms)
opt.run(fmax=0.1, steps=200)

E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
print(f"Relaxed energy (single hydride): {E_relaxed:.3f} eV")
view(complex_hydride_atoms, viewer='x3d')


Hydrides attached to metal: [35, 36]
Removed hydrogen index 36
      Step     Time          Energy          fmax
FIRE:    0 19:46:19  -117019.430100        2.971024
FIRE:    1 19:46:19  -117019.824512        1.914075
FIRE:    2 19:46:19  -117019.859999        2.305626
FIRE:    3 19:46:19  -117019.938330        1.636045
FIRE:    4 19:46:20  -117020.021359        0.706035
FIRE:    5 19:46:20  -117020.044997        0.687528
FIRE:    6 19:46:20  -117020.047683        0.652351
FIRE:    7 19:46:20  -117020.052614        0.599992
FIRE:    8 19:46:20  -117020.059043        0.588830
FIRE:    9 19:46:20  -117020.066127        0.574451
FIRE:   10 19:46:20  -117020.073217        0.557151
FIRE:   11 19:46:20  -117020.080061        0.537326
FIRE:   12 19:46:20  -117020.086877        0.515517
FIRE:   13 19:46:21  -117020.094956        0.489767
FIRE:   14 19:46:21  -117020.105077        0.459945
FIRE:   15 19:46:21  -117020.117690        0.425906
FIRE:   16 19:46:21  -117020.132290        0.387520
FIR

In [618]:
# --- Step 5: get the energy removed ligand ---
opt_ligand = FIRE(ligand_atoms)
ligand_atoms.info.update({"charge": -1})
ligand_atoms.calc = calc
opt_ligand.run(fmax=0.1, steps=200)
E_ligand = ligand_atoms.get_potential_energy()
print(f"Relaxed ligand energy: {E_ligand:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 19:46:31   -37981.434507        8.078212
FIRE:    1 19:46:31   -37982.190084        1.483000
FIRE:    2 19:46:31   -37981.255359       15.149541
FIRE:    3 19:46:31   -37981.994931        5.945813
FIRE:    4 19:46:31   -37982.182059        2.027624
FIRE:    5 19:46:31   -37982.187887        1.891591
FIRE:    6 19:46:32   -37982.198127        1.625251
FIRE:    7 19:46:32   -37982.210247        1.242529
FIRE:    8 19:46:32   -37982.221201        0.791270
FIRE:    9 19:46:32   -37982.228258        0.350596
FIRE:   10 19:46:32   -37982.230159        0.515500
FIRE:   11 19:46:32   -37982.230266        0.506454
FIRE:   12 19:46:32   -37982.230474        0.488625
FIRE:   13 19:46:32   -37982.230776        0.462512
FIRE:   14 19:46:32   -37982.231155        0.428916
FIRE:   15 19:46:32   -37982.231598        0.388975
FIRE:   16 19:46:32   -37982.232084        0.344120
FIRE:   17 19:46:33   -37982.232596        0.296318
FIRE:   18 19:

In [619]:
# Define the H2 molecule (A standard bond length for H2 is ~0.74 Å)
h2_molecule = Atoms("H2", positions=[[0, 0, 0], [0.74, 0, 0]])
h2_molecule.calc = calc

# Relax the H2 molecule
opt_h2 = FIRE(h2_molecule)
# Use a high fmax and fewer steps as H2 relaxation is fast
opt_h2.run(fmax=0.01, steps=100) 

E_H2 = h2_molecule.get_potential_energy()
E_H = 0.5 * E_H2
print(f"Isolated H atom energy: {E_H:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 19:46:37      -31.589716        0.047594
FIRE:    1 19:46:37      -31.589744        0.012353
FIRE:    2 19:46:37      -31.589733        0.031729
FIRE:    3 19:46:37      -31.589737        0.025917
FIRE:    4 19:46:38      -31.589743        0.015339
FIRE:    5 19:46:38      -31.589746        0.001918
Isolated H atom energy: -15.795 eV


In [621]:
# --- Step 6: Compute adsorption energy ---
E_ads = (E_complex_hydride_atoms + E_ligand) - E_complex - E_H
print(f"Adsorption energy of H on Ni: {E_ads:.3f} eV")

Adsorption energy of H on Ni: -0.428 eV


## cyclohexanecarboxylicacid

In [622]:
from ase.io import read
from ase.visualize import view
xyz_path_complex = 'savs_xyz_files/ni_oct_2_cyclohexanecarboxylicacid_3_s_1_conf_1.xyz'
complex_atoms = read(xyz_path_complex)
print(complex_atoms)
# view(complex_atoms, viewer='x3d')

Atoms(symbols='C21H33NiO6', pbc=False)


In [623]:
xyz_path = 'savs_xyz_files/ni_oct_2_cyclohexanecarboxylicacid_2_hydride_2_s_1_conf_1.xyz'
complex_hydride_atoms = read(xyz_path)
print(complex_hydride_atoms)
# view(complex_hydride_atoms, viewer='x3d')

Atoms(symbols='NiOCOC6H11OCOC6H13', pbc=False)


In [624]:
from rdkit import Chem
from rdkit.Chem import AllChem
import ase

smiles = "[O-]C(=O)C1CCCCC1"
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)

# ---- Add 3D coords ----
AllChem.EmbedMolecule(mol, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(mol)

# ---- Your conversion function ----
def rdkit_mol_to_ase_atoms(rdkit_mol):
    conf = rdkit_mol.GetConformer()

    ase_atoms = ase.Atoms(
        numbers=[atom.GetAtomicNum() for atom in rdkit_mol.GetAtoms()],
        positions=conf.GetPositions()
    )
    return ase_atoms
ligand_atoms = rdkit_mol_to_ase_atoms(mol)
print(ligand_atoms)
# view(ligand_atoms, viewer='x3d')

Atoms(symbols='OCOC6H11', pbc=False)


In [625]:
from ase import Atoms
from ase.optimize import FIRE
from ase.constraints import FixAtoms, FixCartesian
from fairchem.core import pretrained_mlip, FAIRChemCalculator
from ase.io import write
import numpy as np

# --- Step 1: Initialize FairChem calculator ---
predictor = pretrained_mlip.get_predict_unit("uma-s-1p1", device="cuda")
calc = FAIRChemCalculator(predictor, task_name="omol")
complex_atoms.calc = calc
# --- Step 2: Relax the complex ---
opt = FIRE(complex_atoms)
opt.run(fmax=0.1, steps=200)
E_complex = complex_atoms.get_potential_energy()
print(f"Relaxed complex energy: {E_complex:.3f} eV")
view(complex_atoms, viewer='x3d')

      Step     Time          Energy          fmax
FIRE:    0 19:47:30   -75634.181705       18.030830
FIRE:    1 19:47:30   -75639.140186        9.986839
FIRE:    2 19:47:30   -75641.059869        3.994994
FIRE:    3 19:47:30   -75641.160575        3.145977
FIRE:    4 19:47:30   -75641.326178        2.780010
FIRE:    5 19:47:30   -75641.560475        2.163353
FIRE:    6 19:47:31   -75641.731902        1.560856
FIRE:    7 19:47:31   -75641.785174        2.091096
FIRE:    8 19:47:31   -75641.797758        1.967772
FIRE:    9 19:47:31   -75641.820199        1.745246
FIRE:   10 19:47:31   -75641.847873        1.435558
FIRE:   11 19:47:31   -75641.875710        1.069406
FIRE:   12 19:47:31   -75641.899834        0.777715
FIRE:   13 19:47:31   -75641.918953        0.814678
FIRE:   14 19:47:31   -75641.934793        0.885267
FIRE:   15 19:47:31   -75641.952577        0.948073
FIRE:   16 19:47:32   -75641.976682        0.932096
FIRE:   17 19:47:32   -75642.010289        0.854140
FIRE:   18 19:

In [626]:
# # --- Step 3: Energy of complex + Hydride ---
# opt_ads = FIRE(complex_hydride_atoms)
# complex_hydride_atoms.calc = calc
# opt_ads.run(fmax=0.1, steps=200)
# E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
# print(f"Relaxed complex + hydride energy: {E_complex_hydride_atoms:.3f} eV")
# view(complex_hydride_atoms, viewer='x3d')

In [627]:
import numpy as np
from ase.optimize import FIRE
from ase.visualize import view

# ----------------------------------------------------------
# 1. Identify the metal atom
# ----------------------------------------------------------
metal_index = 0  # assume metal is atom 0
positions = complex_hydride_atoms.get_positions()
metal_pos = positions[metal_index]

# ----------------------------------------------------------
# 2. Find hydrogens bound to the metal
# ----------------------------------------------------------
hydride_indices = []
for i, atom in enumerate(complex_hydride_atoms):
    if atom.symbol == 'H':
        dist = np.linalg.norm(positions[i] - metal_pos)
        if dist < 2.0:    # typical M–H bond length cutoff
            hydride_indices.append(i)

print("Hydrides attached to metal:", hydride_indices)

# ----------------------------------------------------------
# 3. Remove one hydrogen (if there are two)
# ----------------------------------------------------------
if len(hydride_indices) > 1:
    remove_index = hydride_indices[1]  # remove second hydride
    complex_hydride_atoms.pop(remove_index)
    print(f"Removed hydrogen index {remove_index}")

# ----------------------------------------------------------
# 4. Run optimization with the calculator
# ----------------------------------------------------------
complex_hydride_atoms.calc = calc

opt = FIRE(complex_hydride_atoms)
opt.run(fmax=0.1, steps=200)

E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
print(f"Relaxed energy (single hydride): {E_relaxed:.3f} eV")
view(complex_hydride_atoms, viewer='x3d')

Hydrides attached to metal: [41, 42]
Removed hydrogen index 42
      Step     Time          Energy          fmax
FIRE:    0 19:47:47   -64118.126272       19.891680
FIRE:    1 19:47:47   -64122.420096        7.822965
FIRE:    2 19:47:47   -64123.521556        2.914711
FIRE:    3 19:47:47   -64123.601111        2.212630
FIRE:    4 19:47:47   -64123.700149        1.154042
FIRE:    5 19:47:47   -64123.756077        1.235416
FIRE:    6 19:47:47   -64123.776555        2.023397
FIRE:    7 19:47:47   -64123.786329        1.869782
FIRE:    8 19:47:47   -64123.803589        1.603374
FIRE:    9 19:47:47   -64123.824539        1.310566
FIRE:   10 19:47:48   -64123.845153        1.016892
FIRE:   11 19:47:48   -64123.862566        0.807678
FIRE:   12 19:47:48   -64123.876146        0.754389
FIRE:   13 19:47:48   -64123.887629        0.949322
FIRE:   14 19:47:48   -64123.901205        1.154312
FIRE:   15 19:47:48   -64123.920314        1.225244
FIRE:   16 19:47:48   -64123.947104        1.106694
FIR

In [628]:
# --- Step 5: get the energy removed ligand ---
opt_ligand = FIRE(ligand_atoms)
ligand_atoms.info.update({"charge": -1})
ligand_atoms.calc = calc
opt_ligand.run(fmax=0.1, steps=200)
E_ligand = ligand_atoms.get_potential_energy()
print(f"Relaxed ligand energy: {E_ligand:.3f} eV")
view(ligand_atoms, viewer='x3d')

      Step     Time          Energy          fmax
FIRE:    0 19:47:57   -11533.877813        6.429525
FIRE:    1 19:47:58   -11534.397102        1.867017
FIRE:    2 19:47:58   -11534.021407        9.108618
FIRE:    3 19:47:58   -11534.319658        4.618174
FIRE:    4 19:47:58   -11534.462230        1.464902
FIRE:    5 19:47:58   -11534.464963        1.349604
FIRE:    6 19:47:58   -11534.469687        1.135540
FIRE:    7 19:47:58   -11534.475183        0.856582
FIRE:    8 19:47:58   -11534.480190        0.573276
FIRE:    9 19:47:58   -11534.483831        0.412520
FIRE:   10 19:47:58   -11534.485956        0.482751
FIRE:   11 19:47:59   -11534.487139        0.623980
FIRE:   12 19:47:59   -11534.488298        0.720277
FIRE:   13 19:47:59   -11534.490142        0.724655
FIRE:   14 19:47:59   -11534.492997        0.619154
FIRE:   15 19:47:59   -11534.496445        0.414401
FIRE:   16 19:47:59   -11534.499331        0.307400
FIRE:   17 19:47:59   -11534.500736        0.354242
FIRE:   18 19:

In [629]:
# Define the H2 molecule (A standard bond length for H2 is ~0.74 Å)
h2_molecule = Atoms("H2", positions=[[0, 0, 0], [0.74, 0, 0]])
h2_molecule.calc = calc

# Relax the H2 molecule
opt_h2 = FIRE(h2_molecule)
# Use a high fmax and fewer steps as H2 relaxation is fast
opt_h2.run(fmax=0.01, steps=100)

E_H2 = h2_molecule.get_potential_energy()
E_H = 0.5 * E_H2
print(f"Isolated H atom energy: {E_H:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 19:48:07      -31.589716        0.047594
FIRE:    1 19:48:07      -31.589744        0.012353
FIRE:    2 19:48:07      -31.589733        0.031729
FIRE:    3 19:48:07      -31.589737        0.025917
FIRE:    4 19:48:07      -31.589743        0.015339
FIRE:    5 19:48:07      -31.589746        0.001918
Isolated H atom energy: -15.795 eV


In [631]:
# --- Step 6: Compute adsorption energy ---
E_ads = (E_complex_hydride_atoms + E_ligand) - E_complex - E_H
print(f"Adsorption energy of H on Ni: {E_ads:.3f} eV")

Adsorption energy of H on Ni: -0.456 eV


## 2-mercaptopyridine

In [661]:
from ase.io import read
from ase.visualize import view
xyz_path_complex = 'complexes/xyz/2mercaptopyridine_structure.xyz'
complex_atoms = read(xyz_path_complex)
# view(complex_atoms, viewer='x3d')

In [662]:
xyz_path = 'complexes/xyz/ni_oct_2_2mercaptopyridine_2_hydride_2_s_1_conf_1.xyz'
complex_hydride_atoms = read(xyz_path)
# view(complex_hydride_atoms, viewer='x3d')

In [663]:
from rdkit import Chem
from rdkit.Chem import AllChem
import ase

smiles = "S=C1[N-]C=CC=C1"   # 2,6-difluoropyridine
mol = Chem.MolFromSmiles(smiles)
mol = Chem.AddHs(mol)

# ---- Add 3D coords ----
AllChem.EmbedMolecule(mol, AllChem.ETKDG())
AllChem.UFFOptimizeMolecule(mol)

# ---- Your conversion function ----
def rdkit_mol_to_ase_atoms(rdkit_mol):
    conf = rdkit_mol.GetConformer()

    ase_atoms = ase.Atoms(
        numbers=[atom.GetAtomicNum() for atom in rdkit_mol.GetAtoms()],
        positions=conf.GetPositions()
    )
    return ase_atoms
ligand_atoms = rdkit_mol_to_ase_atoms(mol)
print(ligand_atoms)
# view(ligand_atoms, viewer='x3d')

Atoms(symbols='SCNC4H4', pbc=False)


## Compared to the ligand in the complex, this ligand has an extra hydrogen attached to the Nitrogen

In [664]:
from ase import Atoms
from ase.optimize import FIRE
from ase.constraints import FixAtoms, FixCartesian
from fairchem.core import pretrained_mlip, FAIRChemCalculator
from ase.io import write
import numpy as np

# --- Step 1: Initialize FairChem calculator ---
predictor = pretrained_mlip.get_predict_unit("uma-s-1p1", device="cuda")
calc = FAIRChemCalculator(predictor, task_name="omol")
complex_atoms.calc = calc
# --- Step 2: Relax the complex ---
opt = FIRE(complex_atoms)
opt.run(fmax=0.1, steps=200)
E_complex = complex_atoms.get_potential_energy()
print(f"Relaxed complex energy: {E_complex:.3f} eV")
view(complex_atoms, viewer='x3d')

      Step     Time          Energy          fmax
FIRE:    0 20:17:12   -93767.719408        4.768581
FIRE:    1 20:17:12   -93768.915706        2.202763
FIRE:    2 20:17:12   -93769.004608        3.413840
FIRE:    3 20:17:13   -93769.178994        2.488510
FIRE:    4 20:17:13   -93769.371408        1.079427
FIRE:    5 20:17:13   -93769.444415        0.982358
FIRE:    6 20:17:13   -93769.448733        0.945336
FIRE:    7 20:17:13   -93769.456683        0.873603
FIRE:    8 20:17:13   -93769.467066        0.771911
FIRE:    9 20:17:13   -93769.478553        0.647772
FIRE:   10 20:17:13   -93769.490061        0.511779
FIRE:   11 20:17:13   -93769.501118        0.455704
FIRE:   12 20:17:13   -93769.511947        0.544412
FIRE:   13 20:17:14   -93769.524452        0.635850
FIRE:   14 20:17:14   -93769.539733        0.668142
FIRE:   15 20:17:14   -93769.558490        0.620341
FIRE:   16 20:17:14   -93769.580108        0.483427
FIRE:   17 20:17:14   -93769.602494        0.320320
FIRE:   18 20:

In [665]:
# # --- Step 3: Energy of complex + Hydride ---
# opt_ads = FIRE(complex_hydride_atoms)
# complex_hydride_atoms.calc = calc
# opt_ads.run(fmax=0.1, steps=200)
# E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
# print(f"Relaxed complex + hydride energy: {E_complex_hydride_atoms:.3f} eV")
# view(complex_hydride_atoms, viewer='x3d')

In [666]:
import numpy as np
from ase.optimize import FIRE
from ase.visualize import view

# ----------------------------------------------------------
# 1. Identify the metal atom
# ----------------------------------------------------------
metal_index = 0  # assume metal is atom 0
positions = complex_hydride_atoms.get_positions()
metal_pos = positions[metal_index]

# ----------------------------------------------------------
# 2. Find hydrogens bound to the metal
# ----------------------------------------------------------
hydride_indices = []
for i, atom in enumerate(complex_hydride_atoms):
    if atom.symbol == 'H':
        dist = np.linalg.norm(positions[i] - metal_pos)
        if dist < 2.0:    # typical M–H bond length cutoff
            hydride_indices.append(i)

print("Hydrides attached to metal:", hydride_indices)

# ----------------------------------------------------------
# 3. Remove one hydrogen (if there are two)
# ----------------------------------------------------------
if len(hydride_indices) > 1:
    remove_index = hydride_indices[1]  # remove second hydride
    complex_hydride_atoms.pop(remove_index)
    print(f"Removed hydrogen index {remove_index}")

# ----------------------------------------------------------
# 4. Run optimization with the calculator
# ----------------------------------------------------------
complex_hydride_atoms.calc = calc

opt = FIRE(complex_hydride_atoms)
opt.run(fmax=0.1, steps=200)

E_complex_hydride_atoms = complex_hydride_atoms.get_potential_energy()
print(f"Relaxed energy (single hydride): {E_relaxed:.3f} eV")
view(complex_hydride_atoms, viewer='x3d')


Hydrides attached to metal: [23, 24]
Removed hydrogen index 24
      Step     Time          Energy          fmax
FIRE:    0 20:17:16   -76207.703580        4.764027
FIRE:    1 20:17:16   -76208.731467        2.859679
FIRE:    2 20:17:17   -76208.855209        3.151210
FIRE:    3 20:17:17   -76208.973842        2.280831
FIRE:    4 20:17:17   -76209.107228        0.951885
FIRE:    5 20:17:17   -76209.158394        0.615198
FIRE:    6 20:17:17   -76209.160366        0.589153
FIRE:    7 20:17:17   -76209.163977        0.538786
FIRE:    8 20:17:17   -76209.168650        0.467694
FIRE:    9 20:17:17   -76209.173734        0.386008
FIRE:   10 20:17:17   -76209.178694        0.385620
FIRE:   11 20:17:18   -76209.183288        0.382696
FIRE:   12 20:17:18   -76209.187620        0.375288
FIRE:   13 20:17:18   -76209.192498        0.422582
FIRE:   14 20:17:18   -76209.198431        0.460014
FIRE:   15 20:17:18   -76209.205763        0.434806
FIRE:   16 20:17:18   -76209.214257        0.337668
FIR

In [667]:
# --- Step 5: get the energy removed ligand ---
opt_ligand = FIRE(ligand_atoms)
ligand_atoms.info.update({"charge": -1})
ligand_atoms.calc = calc
opt_ligand.run(fmax=0.1, steps=200)
E_ligand = ligand_atoms.get_potential_energy()
print(f"Relaxed ligand energy: {E_ligand:.3f} eV")
view(ligand_atoms, viewer='x3d')

      Step     Time          Energy          fmax
FIRE:    0 20:17:20   -17576.153641        2.985608
FIRE:    1 20:17:20   -17576.333007        1.165752
FIRE:    2 20:17:21   -17576.292989        2.371566
FIRE:    3 20:17:21   -17576.332193        1.682414
FIRE:    4 20:17:21   -17576.371524        0.764631
FIRE:    5 20:17:21   -17576.377681        0.754379
FIRE:    6 20:17:21   -17576.378984        0.712649
FIRE:    7 20:17:21   -17576.381287        0.632267
FIRE:    8 20:17:21   -17576.384066        0.519665
FIRE:    9 20:17:21   -17576.386736        0.385768
FIRE:   10 20:17:21   -17576.388837        0.253236
FIRE:   11 20:17:22   -17576.390230        0.269513
FIRE:   12 20:17:22   -17576.391168        0.324062
FIRE:   13 20:17:22   -17576.392206        0.402937
FIRE:   14 20:17:22   -17576.393821        0.417147
FIRE:   15 20:17:22   -17576.396242        0.354954
FIRE:   16 20:17:22   -17576.399128        0.224398
FIRE:   17 20:17:22   -17576.401533        0.152287
FIRE:   18 20:

In [668]:
# Define the H2 molecule (A standard bond length for H2 is ~0.74 Å)
h2_molecule = Atoms("H2", positions=[[0, 0, 0], [0.74, 0, 0]])
h2_molecule.calc = calc

# Relax the H2 molecule
opt_h2 = FIRE(h2_molecule)
# Use a high fmax and fewer steps as H2 relaxation is fast
opt_h2.run(fmax=0.01, steps=100) 

E_H2 = h2_molecule.get_potential_energy()
E_H = 0.5 * E_H2
print(f"Isolated H atom energy: {E_H:.3f} eV")

      Step     Time          Energy          fmax
FIRE:    0 20:17:23      -31.589716        0.047594
FIRE:    1 20:17:23      -31.589744        0.012353
FIRE:    2 20:17:23      -31.589733        0.031727
FIRE:    3 20:17:23      -31.589737        0.025917
FIRE:    4 20:17:23      -31.589743        0.015339
FIRE:    5 20:17:23      -31.589746        0.001920
Isolated H atom energy: -15.795 eV


In [670]:
# --- Step 6: Compute adsorption energy ---
E_ads = (E_complex_hydride_atoms + E_ligand) - E_complex - E_H
print(f"Adsorption energy of H on Ni: {E_ads:.3f} eV")

Adsorption energy of H on Ni: -0.067 eV


In [435]:
# Summary of adsorption energies:
{
 'benzimidazole':-0.262,
 '2_mercapto_6_methylpyridine':-0.295,
 '6_mercaptonicotinonitrile':-1.616, # the hydrogens got desorped
 '3_bromo_6_mercaptopyridine':-1.244, # the hydrogens got desorped
 '3_aminopyridine_2_1H_thione':-0.935, # the hydrogens got desorped
 '5_chloropyridine_2_thiol':-0.547,
 "2-mercaptopyridine":-0.214,
 '3_aminopyridine_2_1H_thione':-0.935, # the hydrogens got desorped
    
 "3-5-benzenetricarboxylicacid": -1.650,
 "5,7dichloro8hydroxyquinoline": -1.502,
 "cyclohexanecarboxylicacid": -1.207,
}

{'benzimidazole': -0.262,
 '2_mercapto_6_methylpyridine': -0.295,
 '6_mercaptonicotinonitrile': -1.616,
 '3_bromo_6_mercaptopyridine': -1.244,
 '3_aminopyridine_2_1H_thione': -0.935,
 '5_chloropyridine_2_thiol': -0.547,
 '2-mercaptopyridine': -0.214,
 '3-5-benzenetricarboxylicacid': -1.65,
 '5,7dichloro8hydroxyquinoline': -1.502,
 'cyclohexanecarboxylicacid': -1.207}